# Population

> Data Structures for Population

In [1]:
#| default_exp population

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| export
from dataclasses import field
from typing import List, Optional, Dict, Callable

from flax.struct import dataclass as flax_dataclass # Using flax's dataclass for JAX-friendliness
import jax
import jax.numpy as jnp

from chewc.sp import SimParam
from typing import Tuple

import msprime
import tskit
import numpy as np
import random
from collections import defaultdict
# --- Utility Functions ---

def _pad_and_stack_arrays(arrays: List[jnp.ndarray], pad_value: float = jnp.nan) -> jnp.ndarray:
    """
    Pads a list of JAX arrays to a uniform shape and stacks them.

    This function is critical for handling data where entries can have
    variable lengths, such as genetic maps for different chromosomes. JAX's
    core transformations (like `vmap` for vectorization) require arrays to have
    a uniform shape for efficient batching on accelerators (GPU/TPU). This
    utility converts a Python list of arrays into a single, rectangular
    JAX array that meets this requirement.

    Using `jnp.nan` as the default `pad_value` is a robust choice for
    genetic maps, as it's an unambiguous "missing value" marker.
    """
    if not arrays:
        return jnp.array([])
    max_len = max(arr.shape[0] for arr in arrays)
    padded_arrays = [
        jnp.pad(arr, (0, max_len - arr.shape[0]), 'constant', constant_values=pad_value)
        for arr in arrays
    ]
    return jnp.stack(padded_arrays)


# --- Core Population Structure ---



@flax_dataclass(frozen=True) # Make the class immutable, a JAX best practice
class Population:
    """
    A container for all data related to a population of individuals, designed
    for JAX-based genetic simulations.

    This structure is immutable. All operations that modify a population should
    return a new Population object.

    Attributes:
        geno (jnp.ndarray): A 4D array representing the genotypes of the population.
            Shape: `(nInd, nChr, ploidy, nLoci)`. dtype: `jnp.uint8`.

        id (jnp.ndarray): The primary, user-facing identifier for each individual.
            These IDs may not be contiguous or sorted. Shape: `(nInd,)`.
        iid (jnp.ndarray): The internal, zero-indexed, contiguous identifier.
            Crucial for robust indexing in JAX operations. Shape: `(nInd,)`.
        mother (jnp.ndarray): Array of internal IDs (`iid`) for the mother of each
            individual. A value of -1 indicates no known mother. Shape: `(nInd,)`.
        father (jnp.ndarray): Array of internal IDs (`iid`) for the father of each
            individual. A value of -1 indicates no known father. Shape: `(nInd,)`.
        sex (jnp.ndarray): The sex of each individual, represented numerically
            (e.g., 0 for male, 1 for female). dtype: `jnp.int8`. Shape: `(nInd,)`.

        pheno (jnp.ndarray): The phenotypic values for each individual.
            Shape: `(nInd, nTraits)`.
        fixEff (jnp.ndarray): The value of a fixed effect for each individual,
            often used as an intercept in genomic selection models. Shape: `(nInd,)`.
        
        bv (Optional[jnp.ndarray]): The true breeding values (additive genetic effects)
            for each individual. Shape: `(nInd, nTraits)`.
        dd (Optional[jnp.ndarray]): The true dominance deviations for each individual.
            Shape: `(nInd, nTraits)`.
        aa (Optional[jnp.ndarray]): The true additive-by-additive epistatic deviations
            for each individual. Shape: `(nInd, nTraits)`.

        ebv (Optional[jnp.ndarray]): The estimated breeding values for each
            individual. Shape: `(nInd, nTraits)`.
        gxe (Optional[jnp.ndarray]): Genotype-by-environment interaction effects.
            Shape depends on the specific GxE model.

        misc (Dict): A dictionary for storing miscellaneous, non-JAX-critical
            metadata about individuals. Static.
        miscPop (Dict): A dictionary for storing miscellaneous, non-JAX-critical
            metadata about the entire population. Static.
    """
    # --- Core Genotype Info ---
    geno: jnp.ndarray
    
    # --- Pedigree and Identifiers ---
    id: jnp.ndarray
    iid: jnp.ndarray
    mother: jnp.ndarray
    father: jnp.ndarray
    sex: jnp.ndarray

    # --- Trait and Value Data ---
    pheno: jnp.ndarray
    fixEff: jnp.ndarray
    bv: Optional[jnp.ndarray] = None      # Breeding Value (Additive)
    dd: Optional[jnp.ndarray] = None      # Dominance Deviations
    aa: Optional[jnp.ndarray] = None      # Additive-by-Additive Epistatic Deviations
    
    ebv: Optional[jnp.ndarray] = None
    gxe: Optional[jnp.ndarray] = None

    # --- Metadata ---
    misc: Dict = field(default_factory=dict)
    miscPop: Dict = field(default_factory=dict)

    def __post_init__(self):
        """Validates the consistency of the population data after initialization."""
        n_ind = self.nInd
        # Validate pedigree and identifier shapes
        assert self.iid.shape[0] == n_ind, f"Internal ID array length ({self.iid.shape[0]}) must match number of individuals ({n_ind})."
        assert self.mother.shape[0] == n_ind, f"Mother array length ({self.mother.shape[0]}) must match number of individuals ({n_ind})."
        assert self.father.shape[0] == n_ind, f"Father array length ({self.father.shape[0]}) must match number of individuals ({n_ind})."
        assert self.sex.shape[0] == n_ind, f"Sex array length ({self.sex.shape[0]}) must match number of individuals ({n_ind})."
        
        # Validate trait-related array shapes
        assert self.pheno.shape[0] == n_ind, f"Phenotype (pheno) array length ({self.pheno.shape[0]}) must match number of individuals ({n_ind})."
        assert self.fixEff.shape[0] == n_ind, f"Fixed effect (fixEff) array length ({self.fixEff.shape[0]}) must match number of individuals ({n_ind})."
        
        # Validate optional genetic value components
        if self.bv is not None:
            assert self.bv.shape[0] == n_ind, f"Breeding value (bv) array length ({self.bv.shape[0]}) must match number of individuals ({n_ind})."
        if self.dd is not None:
            assert self.dd.shape[0] == n_ind, f"Dominance deviation (dd) array length ({self.dd.shape[0]}) must match number of individuals ({n_ind})."
        if self.aa is not None:
            assert self.aa.shape[0] == n_ind, f"Epistatic deviation (aa) array length ({self.aa.shape[0]}) must match number of individuals ({n_ind})."
        if self.ebv is not None:
            assert self.ebv.shape[0] == n_ind, f"EBV array length ({self.ebv.shape[0]}) must match number of individuals ({n_ind})."

    @property
    def gv(self) -> jnp.ndarray:
        """
        The total genetic value, calculated by summing the component effects
        (bv, dd, aa). Returns a zero array if no components are defined.
        """
        if self.bv is None:
            return jnp.zeros((self.nInd, 0)) # No traits defined
        
        total_gv = self.bv
        if self.dd is not None:
            total_gv += self.dd
        if self.aa is not None:
            total_gv += self.aa
        return total_gv

    @property
    def nInd(self) -> int:
        """Returns the number of individuals in the population."""
        return self.geno.shape[0]
    
    @property
    def nChr(self) -> int:
        """Returns the number of chromosomes in the population."""
        return self.geno.shape[1]

    @property
    def nTraits(self) -> int:
        """Returns the number of traits, inferred from the breeding value (bv) shape."""
        if self.bv is None or self.bv.ndim <= 1:
            return 0
        return self.bv.shape[1]
    
    @property
    def haplo_matrix(self) -> jnp.ndarray:
        """ returns a haplotype matrix of shape (n_ind*ploidy, n_chr*n_markers)"""
        return self.geno.transpose(0, 2, 1, 3).reshape(self.geno.shape[0] * self.geno.shape[2], -1)

    @property
    def dosage(self) -> jnp.ndarray:
        """
        Calculates the dosage of alternate alleles for each individual.

        The dosage is the sum of alleles across the ploidy dimension, resulting
        in a 2D matrix where each entry represents the count of the alternate
        allele at a specific locus for an individual.

        Returns:
            A JAX array of shape `(nInd, nLoci)`, where nLoci is the total
            number of loci across all chromosomes.
        """
        # Sum over the ploidy axis (axis=2) to get dosage per chromosome
        # Shape: (nInd, nChr, nLoci_per_chr)
        dosage_per_chr = jnp.sum(self.geno, axis=2)

        # Reshape to combine the chromosome and loci dimensions
        # Shape: (nInd, nChr * nLoci_per_chr)
        return dosage_per_chr.reshape(self.nInd, -1)

    def __repr__(self) -> str:
        """Provides a concise representation of the Population object."""
        return (f"Population(nInd={self.nInd}, nTraits={self.nTraits}, "
                f"has_ebv={'Yes' if self.ebv is not None else 'No'})")    

# --- Factory Functions ---

# NOTE: This function assumes a uniform number of segregating sites across chromosomes,
def quick_haplo(key: jax.random.PRNGKey, sim_param: 'SimParam', n_ind: int, inbred: bool = False) -> Population:
    """
    Creates a new population with randomly generated haplotypes, analogous to
    AlphaSimR's `quickHaplo` function.

    The structure of the genome (chromosomes, loci, ploidy) is defined by
    the provided SimParam object.

    Args:
        key: A JAX random key. Must be provided by the user to ensure
             reproducibility.
        sim_param: A SimParam object that defines the genome structure.
        n_ind: Number of individuals to create.
        inbred: If True, individuals will be fully inbred (homozygous at all loci).

    Returns:
        A new Population object with random founder individuals.
    """
    key, geno_key, sex_key = jax.random.split(key, 3)

    # Get genome parameters from the SimParam object
    n_chr = sim_param.n_chr
    # We assume uniform number of loci for this simple factory, matching original quickHaplo
    seg_sites = sim_param.gen_map.shape[1] 
    ploidy = sim_param.ploidy

    # Generate random haplotypes for all individuals and chromosomes
    # Shape: (nInd, nChr, ploidy, nLoci)
    if inbred:
        # Generate one set of haplotypes and tile it across the ploidy axis
        base_haplotypes = jax.random.randint(geno_key, (n_ind, n_chr, 1, seg_sites), 0, 2, dtype=jnp.uint8)
        geno = jnp.tile(base_haplotypes, (1, 1, ploidy, 1))
    else:
        # Fully random, outbred individuals
        geno = jax.random.randint(geno_key, (n_ind, n_chr, ploidy, seg_sites), 0, 2, dtype=jnp.uint8)

    # --- Create Pedigree and IDs using JAX arrays ---
    ids = jnp.arange(n_ind)
    sex_array = jax.random.choice(sex_key, jnp.array([0, 1], dtype=jnp.int8), (n_ind,))

    return Population(
        geno=geno,
        id=ids,
        iid=ids,  # In a new pop, id and iid are the same
        mother=jnp.full(n_ind, -1, dtype=jnp.int32),
        father=jnp.full(n_ind, -1, dtype=jnp.int32),
        sex=sex_array,
        pheno=jnp.zeros((n_ind, 0)),
        fixEff=jnp.zeros(n_ind, dtype=jnp.float32), # Default fixed effect of 0
        bv=jnp.zeros((n_ind, 0)),  # No traits by default
        dd=None,
        aa=None,

    )



def msprime_pop(key: jax.random.PRNGKey, n_ind: int, n_loci_per_chr: int, n_chr:int, ploidy = 2) -> Tuple[Population, jnp.ndarray]:

    """
    Creates a new founder population using a standard msprime coalescent simulation.

    This function provides a biologically more realistic alternative to `quick_haplo`
    by generating genotypes and a genetic map based on population genetics principles
    (e.g., effective population size, mutation, recombination).

    Args:
        key: A JAX random key, used to derive a seed for msprime's random number generator.
        sim_param: A SimParam object that defines the genome structure (ploidy, n_chr).
                   The chromosome lengths are hardcoded in this version.
        n_ind: The number of founder individuals to generate and return.
        n_loci_per_chr: The number of SNPs (loci) to select for each chromosome.

    Returns:
        A new Population object containing the simulated founder individuals. The
        generated genetic map (in cM) is stored in the `miscPop` dictionary.
    """
    # --- Hardcoded Simulation Parameters ---
    # These define the ancestral population from which founders are drawn.
    num_simulated_individuals = 500
    effective_population_size = 10_000
    mutation_rate = 2e-8
    chromosome_lengths = [1_500_000, 1_000_000, 500_000] # Standard lengths
    recombination_rate_per_chr = 2e-8
    maf_threshold = 0.1
    # --- End of Hardcoded Parameters ---

    # Derive a simple integer seed for msprime from the JAX key
    random_seed = int(jnp.sum(key))

    if n_ind > num_simulated_individuals:
        raise ValueError(f"Number of founders requested ({n_ind}) cannot exceed the base simulated population size ({num_simulated_individuals}).")

    # --- Run msprime Simulation ---
    num_haplotypes = num_simulated_individuals * ploidy
    
    # Create the recombination map for msprime
    rate_map_positions = [0] + list(np.cumsum(chromosome_lengths))
    rate_map_rates = [recombination_rate_per_chr] * len(chromosome_lengths)
    rate_map = msprime.RateMap(position=rate_map_positions, rate=rate_map_rates)

    ts = msprime.sim_ancestry(
        samples=num_haplotypes, population_size=effective_population_size,
        recombination_rate=rate_map, random_seed=random_seed
    )
    mts = msprime.sim_mutations(ts, rate=mutation_rate, random_seed=random_seed)

    # --- Robust Data Extraction ---
    true_num_individuals = mts.num_samples // ploidy
    all_variants = list(mts.variants())

    # Initialize final data structures
    genetic_map = np.full((n_chr, n_loci_per_chr), np.nan)
    full_haplotype_matrix = np.full(
        (true_num_individuals, n_chr, ploidy, n_loci_per_chr), np.nan
    )

    for i in range(n_chr):
        chr_start, chr_end, recomb_rate = rate_map.left[i], rate_map.right[i], rate_map.rate[i]

        eligible_snps = [
            var for var in all_variants
            if chr_start <= var.site.position < chr_end and len(var.alleles) == 2 and min(np.mean(var.genotypes), 1 - np.mean(var.genotypes)) > maf_threshold
        ]
        
        num_found = len(eligible_snps)
        num_to_select = min(num_found, n_loci_per_chr)
        
        if num_to_select > 0:
            selected_snps = random.sample(eligible_snps, num_to_select)
            selected_snps.sort(key=lambda v: v.site.position)

            for snp_idx, snp in enumerate(selected_snps):
                genotypes_for_snp = snp.genotypes.reshape(true_num_individuals, ploidy)
                full_haplotype_matrix[:, i, :, snp_idx] = genotypes_for_snp
            
            positions_cm = [(v.site.position - chr_start) * recomb_rate * 100 for v in selected_snps]
            genetic_map[i, :num_to_select] = positions_cm

    # --- Sample Founders and Format for Population Object ---
    founder_indices = np.sort(random.sample(range(true_num_individuals), n_ind))
    founder_haplotypes = full_haplotype_matrix[founder_indices, :, :, :]
    
    # Convert to JAX arrays
    geno = jnp.array(founder_haplotypes, dtype=jnp.uint8)
    
    # --- Create Pedigree and IDs using JAX arrays ---
    ids = jnp.arange(n_ind)
    key, sex_key = jax.random.split(key)
    sex_array = jax.random.choice(sex_key, jnp.array([0, 1], dtype=jnp.int8), (n_ind,))

    return (Population(
        geno=geno,
        id=ids,
        iid=ids,
        mother=jnp.full(n_ind, -1, dtype=jnp.int32),
        father=jnp.full(n_ind, -1, dtype=jnp.int32),
        sex=sex_array,
        pheno=jnp.zeros((n_ind, 0)),
        fixEff=jnp.zeros(n_ind, dtype=jnp.float32),
        bv=jnp.zeros((n_ind, 0)),
        dd=None,
        aa=None,
    ),
    jnp.array(genetic_map)
    )



In [4]:
#| hide
import nbdev; nbdev.nbdev_export()